In [1]:
# Import useful packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.sparse import csr_array
from scipy.sparse import lil_array
from scipy.sparse import save_npz
from scipy.sparse import load_npz
import os
from datetime import datetime, timedelta

In [3]:
# Read in the data

# Check which server we're on (in case the data is in different places on different servers)
import socket
hostname = socket.gethostname()

# Get paths to data
if hostname == "blpc1" or hostname == "blpc2":
    data_path = "/datax/scratch/nstieg/"
elif hostname == "cosmic-gpu-1":
    data_path = "/mnt/cosmic-gpu-1/data0/nstiegle/"
else:
    raise Exception("Data path not known")

full_dataset_path = data_path + "25GHz_higher.pkl"
coherent_dataset_path = data_path + "25GHz_higher_coherent.pkl"
incoherent_dataset_path = data_path + "25GHz_higher_incoherent.pkl"

# Read in data
# coherent = pd.read_pickle(coherent_dataset_path)
# incoherent = pd.read_pickle(incoherent_dataset_path)
df = pd.read_pickle(full_dataset_path)

In [5]:
# Check what the extensions of the file_uri's are in the full dataset. Are any of them the actual stamps?
for file_uri in df["file_uri"].tolist():
    filename = file_uri.split('/')[-1]
    extension = filename.split('.')[-1]
    if extension != "hits":
        print(file_uri)

In [12]:
# Let's get all the hits which correspond to one stamp file
first_stamp = df["file_uri"][0]
in_first_stamp = df[df["file_uri"] == first_stamp].reset_index(drop=True)
in_first_stamp.shape
in_first_stamp.to_csv("/home/nstiegle/BL-COSMIC-2024-proj/stamps/hits_in_first_stamp.csv", index=False)